# Descarga de Dados GFS


Trabajaremos con el utilitario [siphon](https://unidata.github.io/siphon/latest/index.html), que fue diseñado para tener acceso directo a al servidor [THREDDS](https://www.unidata.ucar.edu/software/tds/current/)

Para utilizar el `siphon` debemos tener en consideración lo siguiente:
- Python >= 3.5
- numpy >= 1.8
- pandas

Y otras informaciones que podemos encontrar en el manual de instalación de la herramienta.


## Instalación de librerias necesarias

- Librería siphon


```bash
conda install -c conda-forge siphon xarray[complete] netCDF4
```

>El `netCDF4` será instalado para ser utilizado como background del siphon a la hora de hacer los requerimientos \
En este caso iremos a instalar el `xarray` para tener un control facilitado de los archivos `netcdf`



La lista completa de datos como medio grado entra en este [link](https://thredds.ucar.edu/thredds/catalog/catalog.html)

In [1]:
from siphon.catalog import TDSCatalog

# si el catalogo que se desea es de 0.5 grados
# best_gfs = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p5deg/catalog.xml')

# si el catalogo que se desea es de 0.25 grados
best_gfs = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p25deg/catalog.xml')

# lista de colecciones
best_gfs.datasets

['Full Collection (Reference / Forecast Time) Dataset', 'Best GFS Quarter Degree Forecast Time Series', 'Latest Collection for GFS Quarter Degree Forecast']

## Verificando datos disponibles

In [2]:
best_disp = list(best_gfs.datasets.values())[0]
ncss = best_disp.subset()
query = ncss.query()
for lista in ncss.variables:
    print(lista)

Geopotential_height_maximum_wind
Ventilation_Rate_planetary_boundary
Cloud_water_entire_atmosphere_single_layer
Pressure_middle_cloud_top_Mixed_intervals_Average
Reflectivity_height_above_ground
Convective_precipitation_rate_surface
Specific_humidity_height_above_ground
u-component_of_wind_sigma
Best_4_layer_Lifted_Index_surface
Upward_Short-Wave_Radiation_Flux_atmosphere_top_Mixed_intervals_Average
v-component_of_wind_tropopause
Vertical_velocity_geometric_isobaric
Cloud_mixing_ratio_isobaric
Ozone_Mixing_Ratio_isobaric
Latent_heat_net_flux_surface_Mixed_intervals_Average
Wilting_Point_surface
Precipitable_water_entire_atmosphere_single_layer
Categorical_Snow_surface_Mixed_intervals_Average
Ice_cover_surface
Relative_humidity_sigma_layer
Vertical_Speed_Shear_potential_vorticity_surface
Maximum_temperature_height_above_ground_Mixed_intervals_Maximum
Geopotential_height_zeroDegC_isotherm
Planetary_Boundary_Layer_Height_surface
Downward_Long-Wave_Radp_Flux_surface_Mixed_intervals_Average

In [3]:
best_ds = list(best_gfs.datasets.values())[1]

# diccionario de urls disponibles con la información
best_ds.access_urls

{'OPENDAP': 'https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/GFS/Global_0p25deg/Best',
 'CdmRemote': 'https://thredds.ucar.edu/thredds/cdmremote/grib/NCEP/GFS/Global_0p25deg/Best',
 'JupyterNotebook': 'https://thredds.ucar.edu/thredds/notebook/grib/NCEP/GFS/Global_0p25deg/Best',
 'NetcdfSubset': 'https://thredds.ucar.edu/thredds/ncss/grid/grib/NCEP/GFS/Global_0p25deg/Best',
 'WMS': 'https://thredds.ucar.edu/thredds/wms/grib/NCEP/GFS/Global_0p25deg/Best',
 'WCS': 'https://thredds.ucar.edu/thredds/wcs/grib/NCEP/GFS/Global_0p25deg/Best',
 'ISO': 'https://thredds.ucar.edu/thredds/iso/grib/NCEP/GFS/Global_0p25deg/Best',
 'NCML': 'https://thredds.ucar.edu/thredds/ncml/grib/NCEP/GFS/Global_0p25deg/Best',
 'UDDC': 'https://thredds.ucar.edu/thredds/uddc/grib/NCEP/GFS/Global_0p25deg/Best'}

In [4]:
# utilizando el servicio de netcdf-subset

from siphon.ncss import NCSS
ncss = NCSS(best_ds.access_urls['NetcdfSubset'])

In [5]:
# generando un sistema de consulta en el netcdf-subset
query = ncss.query()

Localización de coordenadas geográficas normal         |   Como es interpretado por el `siphon`
:----------------------------------------------:|:---------------------------------------:
<img src="./figs/globo.png" width="550"/>      | <img src="./figs/globo_m.png" width="550"/>


In [6]:
from datetime import datetime

now = datetime.utcnow()

# buscando la posición de lima
query.lonlat_box(west=270, north=0, east=300, south=-20).vertical_level(100_000).time(now)
query.accept('netcdf4')
query.variables('Temperature_isobaric', 'Relative_humidity_isobaric')

var=Relative_humidity_isobaric&var=Temperature_isobaric&time=2022-10-26T21%3A58%3A01.453163&west=270&east=300&south=-20&north=0&vertCoord=100000&accept=netcdf4

In [7]:
data = ncss.get_data(query)
list(data.variables.keys())

['time1',
 'latitude',
 'reftime1',
 'isobaric',
 'longitude',
 'Relative_humidity_isobaric',
 'Temperature_isobaric',
 'LatLon_721X1440-0p13S-180p00E-2']

In [8]:
100_000

100000

In [9]:
from datetime import datetime
print(datetime.now().month)
print(dir(datetime))

10
['__add__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__rsub__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', 'astimezone', 'combine', 'ctime', 'date', 'day', 'dst', 'fold', 'fromisocalendar', 'fromisoformat', 'fromordinal', 'fromtimestamp', 'hour', 'isocalendar', 'isoformat', 'isoweekday', 'max', 'microsecond', 'min', 'minute', 'month', 'now', 'replace', 'resolution', 'second', 'strftime', 'strptime', 'time', 'timestamp', 'timetuple', 'timetz', 'today', 'toordinal', 'tzinfo', 'tzname', 'utcfromtimestamp', 'utcnow', 'utcoffset', 'utctimetuple', 'weekday', 'year']


## Retornar al [indice](./../indice.md)